In [2]:
# !rm -rf __MACOSX/news
# !rm -rf news
# !rm news.zip
# !wget https://f000.backblazeb2.com/file/malay-dataset/news/news.zip
# !unzip news.zip

# !rm -rf __MACOSX
# !rm news-30k.json.zip news-30k.json
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/news/news-30k/news-30k.json.zip
# !unzip news-30k.json.zip

In [6]:
import re
from unidecode import unidecode
from malaya.text.rules import normalized_chars

def filter_news(string):
    string = string.lower()
    return 'javascript is disabled' in string or 'requires javascript' in string or 'javascript' in string \
    or 'président' in string

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def transformer_textcleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    string = [w.title() if w[0].isupper() else w for w in string]
    return ' '.join(string)

import re

def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [9]:
from glob import glob
from tqdm import tqdm
import json

news = glob('news/*.json')
len(news)

before, after = [], []
for n in tqdm(news):
    with open(n) as fopen:
        data = json.load(fopen)
    for i in data:
        if not filter_news(i['text']) and i['language'] != 'ENGLISH' and len(i['text']) and len(i['title']):
            before.append(cleaning(i['text']))
            after.append(cleaning(i['title']))
            
with open('news-30k.json') as fopen:
    data = json.load(fopen)
    
for i in data:
    if not filter_news(i['text']) and i['language'] == 'malay' and len(i['text']) and len(i['title']):
        before.append(cleaning(i['text']))
        after.append(cleaning(i['title']))
            
len(before), len(after)

100%|██████████| 860/860 [00:16<00:00, 51.68it/s]


(120410, 120410)

In [11]:
before[0], after[0]

('KalbarOnline, Sekadau – Ikatan alumni bintara polisi angkatan ke-27 tahun 2005 gelombang II (ZLD) Polres Sekadau, melakukan aksi bakti sosial dengan membagikan paket sembako kepada warga kurang mampu dan panti asuhan. Sebanyak 25 paket sembako berisi beras, gula, tepung, minyak goreng dan mie instan diserahkan langsung secara door to door kepada warga yang sebelumnya telah didata, untuk menghindari kerumunan. Sasarannya adalah warga kurang mampu atau fakir miskin, lansia, warga terdampak pandemi covid-19 dan anak yatim piatu di panti asuhan Harapan Bunda dan Yayasan Filipi. Bripka Hisbullah Aji selaku koordinator aksi baksos mengatakan, ide pemberian bantuan sembako muncul dari rasa keprihatinan terhadap warga lantaran sulitnya ekonomi ditengah masa pandemi. Terlebih bagi warga yang belum menerima bantuan dari pemerintah. Anggota ZLD Polres Sekadau yang berjumlah 22 personel kemudian mengadakan iuran sukarela yang dikumpulkan sehingga aksi bakti sosial ini dapat terealisasi. Bahkan a

In [ ]:
with open('news-data.json', 'w') as fopen:
    json.dump({'before': before, 'after': after}, fopen)